In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout ,Input 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


# Animal recognition


In [2]:
train_dir = 'animals/train'
validation_dir = 'animals/test'

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Apply data augmentation to the training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Split for validation
)

# Validation data should not be augmented
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load and preprocess the datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 1152 images belonging to 24 classes.
Found 80 images belonging to 24 classes.


In [12]:
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
import os
# Verify number of classes
num_classes = train_generator.num_classes
print(f'Number of classes: {num_classes}')

# Pre-trained model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Fine-tune the model
base_model.trainable = True
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Function to build the model (for hyperparameter tuning)
def build_model(hp):
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(hp.Int('units', min_value=128, max_value=512, step=64), activation='relu'),
        Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Hyperparameter tuning
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='animal_recognition'
)

# Perform the search
tuner.search(train_generator, validation_data=validation_generator, epochs=10)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Further train the best model
best_model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Evaluate the best model
animal_test_datagen = ImageDataGenerator(rescale=1./255)
animal_test_generator = animal_test_datagen.flow_from_directory(validation_dir, target_size=(128, 128), batch_size=32, class_mode='categorical')

test_loss, test_accuracy = best_model.evaluate(animal_test_generator)
print(f'Best Animal Recognition Model Test Accuracy: {test_accuracy:.2f}')
print(f'Best Animal Recognition Model Test Loss: {test_loss:.2f}')


Trial 10 Complete [00h 55m 14s]
val_accuracy: 0.9312500059604645

Best val_accuracy So Far: 0.9312500059604645
Total elapsed time: 03h 19m 45s


C:\Users\admin\AppData\Roaming\Python\Python311\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 118 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 1/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 50s 815ms/step - accuracy: 0.9913 - loss: 0.1846 - val_accuracy: 0.9500 - val_loss: 0.3196
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 31s 754ms/step - accuracy: 0.9960 - loss: 0.0366 - val_accuracy: 0.9250 - val_loss: 0.3487
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 31s 778ms/step - accuracy: 0.9959 - loss: 0.0322 - val_accuracy: 0.9375 - val_loss: 0.3600
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 31s 747ms/step - accuracy: 0.9958 - loss: 0.0216 - val_accuracy: 0.9125 - val_loss: 0.3383
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 31s 764ms/step - accuracy: 0.9917 - loss: 0.0299 - val_accuracy: 0.9125 - val_loss: 0.3964
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 31s 748ms/step - accuracy: 0.9931 - loss: 0.0243 - val_accuracy: 0.9125 - val_loss: 0.3807
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 31s 758ms/step - accuracy: 0.9967 - loss: 0.0130 - val_accuracy: 0.9000 - val_loss: 0.4027
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 32s 785ms/step - accuracy: 0.9993 - loss: 0.0096 - val_accu

C:\Users\admin\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 617ms/step - accuracy: 0.9654 - loss: 0.1098
Best Animal Recognition Model Test Accuracy: 0.97
Best Animal Recognition Model Test Loss: 0.14


# Animal Emotion

In [13]:
emotion_train_dir = 'Master Folder/train'
emotion_validation_dir = 'Master Folder/test'

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Apply data augmentation to the training data
emotion_train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Split for validation
)

# Validation data should not be augmented
emotion_val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load and preprocess the datasets
emotion_train_generator = emotion_train_datagen.flow_from_directory(
    emotion_train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

emotion_validation_generator = emotion_val_datagen.flow_from_directory(
    emotion_validation_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 800 images belonging to 4 classes.
Found 7 images belonging to 4 classes.


In [26]:
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
import os
# Verify number of classes
emotion_num_classes = emotion_train_generator.num_classes
print(f'Number of classes: {num_classes}')

# Pre-trained model
emotion_base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Fine-tune the model
emotion_base_model.trainable = True
fine_tune_at = 100
for layer in emotion_base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Function to build the model (for hyperparameter tuning)
def emotion_build_model(hp):
    emotion_model = Sequential([
        emotion_base_model,
        GlobalAveragePooling2D(),
        Dense(hp.Int('units', min_value=128, max_value=512, step=64), activation='relu'),
        Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)),
        Dense(emotion_num_classes, activation='softmax')
    ])
    
    emotion_model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return emotion_model

# Hyperparameter tuning
tuner = RandomSearch(
    emotion_build_model,
    objective='val_accuracy',
    max_trials=15,
    executions_per_trial=2,
    directory='my_dir',
    project_name='Emotion_recognition'
)

# Perform the search
tuner.search(emotion_train_generator, validation_data=emotion_validation_generator, epochs=10)

# Get the best model
emotion_best_model = tuner.get_best_models(num_models=1)[0]

# Further train the best model
emotion_best_model.fit(emotion_train_generator, validation_data=emotion_validation_generator, epochs=10)

# Save the best model
emotion_best_model.save('best_emotion_model.keras')

# Evaluate the best model
emotion_test_datagen = ImageDataGenerator(rescale=1./255)
emotion_test_generator = emotion_test_datagen.flow_from_directory(emotion_validation_dir, target_size=(128, 128), batch_size=32, class_mode='categorical')

test_loss, test_accuracy = emotion_best_model.evaluate(emotion_test_generator)
print(f'Best Emotion Recognition Model Test Accuracy: {test_accuracy:.2f}')
print(f'Best Emotion Recognition Model Test Loss: {test_loss:.2f}')


Trial 15 Complete [00h 09m 23s]
val_accuracy: 0.5000000149011612

Best val_accuracy So Far: 0.5000000149011612
Total elapsed time: 02h 45m 35s
Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 80s 861ms/step - accuracy: 0.4228 - loss: 1.3189 - val_accuracy: 0.5714 - val_loss: 1.4525
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 712ms/step - accuracy: 0.4773 - loss: 1.2333 - val_accuracy: 0.5714 - val_loss: 1.4594
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 698ms/step - accuracy: 0.4791 - loss: 1.2084 - val_accuracy: 0.5714 - val_loss: 1.4460
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 19s 672ms/step - accuracy: 0.4702 - loss: 1.1839 - val_accuracy: 0.5714 - val_loss: 1.4312
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 18s 631ms/step - accuracy: 0.5384 - loss: 1.1265 - val_accuracy: 0.5714 - val_loss: 1.4275
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 17s 612ms/step - accuracy: 0.4995 - loss: 1.1359 - val_accuracy: 0.5714 - val_loss: 1.4301
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 598ms/step - accuracy: 0.5601 - loss: 1.0

In [21]:
best_model.save('best_animal_recognition_model.keras')